# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 28 2022 9:59AM,247464,10,0086018078,ISDIN Corporation,Released
1,Sep 28 2022 9:59AM,247464,10,0086018080,ISDIN Corporation,Released
2,Sep 28 2022 9:59AM,247464,10,0086018081,ISDIN Corporation,Released
3,Sep 28 2022 9:59AM,247464,10,0086018083,ISDIN Corporation,Released
4,Sep 28 2022 9:59AM,247464,10,0086018085,ISDIN Corporation,Released
5,Sep 28 2022 9:59AM,247464,10,0086018088,ISDIN Corporation,Released
6,Sep 28 2022 9:59AM,247464,10,0086018091,ISDIN Corporation,Released
7,Sep 28 2022 9:59AM,247464,10,0086018092,ISDIN Corporation,Released
8,Sep 28 2022 9:59AM,247459,10,PRF-38044,Bio-PRF,Released
9,Sep 28 2022 9:59AM,247459,10,PRF-38048,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,247459,Released,10
26,247460,Released,21
27,247461,Released,16
28,247462,Executing,1
29,247464,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247459,NaN,NaN,10.0
247460,NaN,NaN,21.0
247461,NaN,NaN,16.0
247462,NaN,1.0,NaN
247464,NaN,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247385,16.0,2.0,4.0
247386,0.0,0.0,1.0
247403,9.0,6.0,14.0
247404,4.0,8.0,2.0
247410,14.0,13.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247385,16,2,4
247386,0,0,1
247403,9,6,14
247404,4,8,2
247410,14,13,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247385,16,2,4
1,247386,0,0,1
2,247403,9,6,14
3,247404,4,8,2
4,247410,14,13,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247385,16,2,4
1,247386,,,1
2,247403,9,6,14
3,247404,4,8,2
4,247410,14,13,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 28 2022 9:59AM,247464,10,ISDIN Corporation
8,Sep 28 2022 9:59AM,247459,10,Bio-PRF
18,Sep 28 2022 9:53AM,247462,19,"AdvaGen Pharma, Ltd"
19,Sep 28 2022 9:51AM,247461,15,"Virtus Pharmaceuticals, LLC"
35,Sep 28 2022 9:43AM,247460,15,"Brookfield Pharmaceuticals, LLC"
56,Sep 28 2022 9:35AM,247458,10,"Citieffe, Inc."
58,Sep 28 2022 9:34AM,247457,20,"ACI Healthcare USA, Inc."
59,Sep 28 2022 9:30AM,247435,15,"Alliance Pharma, Inc."
90,Sep 28 2022 9:28AM,247454,10,ISDIN Corporation
99,Sep 28 2022 9:28AM,247450,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 28 2022 9:59AM,247464,10,ISDIN Corporation,,,8
1,Sep 28 2022 9:59AM,247459,10,Bio-PRF,,,10
2,Sep 28 2022 9:53AM,247462,19,"AdvaGen Pharma, Ltd",,1,
3,Sep 28 2022 9:51AM,247461,15,"Virtus Pharmaceuticals, LLC",,,16
4,Sep 28 2022 9:43AM,247460,15,"Brookfield Pharmaceuticals, LLC",,,21
5,Sep 28 2022 9:35AM,247458,10,"Citieffe, Inc.",,,2
6,Sep 28 2022 9:34AM,247457,20,"ACI Healthcare USA, Inc.",,,1
7,Sep 28 2022 9:30AM,247435,15,"Alliance Pharma, Inc.",,,31
8,Sep 28 2022 9:28AM,247454,10,ISDIN Corporation,,,9
9,Sep 28 2022 9:28AM,247450,10,ISDIN Corporation,,,43


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 28 2022 9:59AM,247464,10,ISDIN Corporation,8,,
1,Sep 28 2022 9:59AM,247459,10,Bio-PRF,10,,
2,Sep 28 2022 9:53AM,247462,19,"AdvaGen Pharma, Ltd",,1,
3,Sep 28 2022 9:51AM,247461,15,"Virtus Pharmaceuticals, LLC",16,,
4,Sep 28 2022 9:43AM,247460,15,"Brookfield Pharmaceuticals, LLC",21,,
5,Sep 28 2022 9:35AM,247458,10,"Citieffe, Inc.",2,,
6,Sep 28 2022 9:34AM,247457,20,"ACI Healthcare USA, Inc.",1,,
7,Sep 28 2022 9:30AM,247435,15,"Alliance Pharma, Inc.",31,,
8,Sep 28 2022 9:28AM,247454,10,ISDIN Corporation,9,,
9,Sep 28 2022 9:28AM,247450,10,ISDIN Corporation,43,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 28 2022 9:59AM,247464,10,ISDIN Corporation,8,,
1,Sep 28 2022 9:59AM,247459,10,Bio-PRF,10,,
2,Sep 28 2022 9:53AM,247462,19,"AdvaGen Pharma, Ltd",,1,
3,Sep 28 2022 9:51AM,247461,15,"Virtus Pharmaceuticals, LLC",16,,
4,Sep 28 2022 9:43AM,247460,15,"Brookfield Pharmaceuticals, LLC",21,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 28 2022 9:59AM,247464,10,ISDIN Corporation,8.0,NaN,NaN
1,Sep 28 2022 9:59AM,247459,10,Bio-PRF,10.0,NaN,NaN
2,Sep 28 2022 9:53AM,247462,19,"AdvaGen Pharma, Ltd",NaN,1.0,NaN
3,Sep 28 2022 9:51AM,247461,15,"Virtus Pharmaceuticals, LLC",16.0,NaN,NaN
4,Sep 28 2022 9:43AM,247460,15,"Brookfield Pharmaceuticals, LLC",21.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 28 2022 9:59AM,247464,10,ISDIN Corporation,8.0,0.0,0.0
1,Sep 28 2022 9:59AM,247459,10,Bio-PRF,10.0,0.0,0.0
2,Sep 28 2022 9:53AM,247462,19,"AdvaGen Pharma, Ltd",0.0,1.0,0.0
3,Sep 28 2022 9:51AM,247461,15,"Virtus Pharmaceuticals, LLC",16.0,0.0,0.0
4,Sep 28 2022 9:43AM,247460,15,"Brookfield Pharmaceuticals, LLC",21.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2226971,123.0,8.0,4.0
15,742356,68.0,0.0,0.0
16,247430,0.0,1.0,0.0
19,1237163,17.0,14.0,14.0
20,989631,20.0,8.0,25.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2226971,123.0,8.0,4.0
1,15,742356,68.0,0.0,0.0
2,16,247430,0.0,1.0,0.0
3,19,1237163,17.0,14.0,14.0
4,20,989631,20.0,8.0,25.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,123.0,8.0,4.0
1,15,68.0,0.0,0.0
2,16,0.0,1.0,0.0
3,19,17.0,14.0,14.0
4,20,20.0,8.0,25.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,123.0
1,15,Released,68.0
2,16,Released,0.0
3,19,Released,17.0
4,20,Released,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Completed,4.0,0.0,0.0,14.0,25.0
Executing,8.0,0.0,1.0,14.0,8.0
Released,123.0,68.0,0.0,17.0,20.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Completed,4.0,0.0,0.0,14.0,25.0
1,Executing,8.0,0.0,1.0,14.0,8.0
2,Released,123.0,68.0,0.0,17.0,20.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Completed,4.0,0.0,0.0,14.0,25.0
1,Executing,8.0,0.0,1.0,14.0,8.0
2,Released,123.0,68.0,0.0,17.0,20.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()